In [1]:
# Firstly, some imports

# Jupyter notebook related
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Basics of Python data handling and visualization
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd
from shapely.geometry import Polygon, Point

# Basics of GIS
import geopandas as gpd

# The core of this example
from eolearn.core import EOTask, EOPatch, LinearWorkflow, FeatureType, OverwritePermission, LoadFromDisk, SaveToDisk

# Misc
import pickle
import sys
import os
import datetime
import itertools
from tqdm import tqdm_notebook as tqdm
import enum

In [2]:
path_out = '.\\eopatches\\'
eopatches = []
for i in range(9):
    eopatches.append(EOPatch.load('{}eopatch_{}'.format(path_out, i), lazy_loading=True))

In [3]:
not_ex = []
for i in range(len(eopatches)):
    eopatch = eopatches[i]
    list_date = []
    date_ = datetime.datetime(2018, 5, 6, 0, 0, 1).date()
    for x in eopatch.timestamp:
        list_date.append(x.date())
    try:
        index = list_date.index(date_)
        print(index)
    except:
        print('does not exist')
        not_ex.append(i)
        
not_ex.reverse()
for i in not_ex:
    print('---',i)
    del eopatches[i]

14
8
6
does not exist
does not exist
10
15
10
10
--- 4
--- 3


In [5]:
%%time
_weather = eopatches[0].vector_timeless['weather']
response = int(input('Write 0 if you want to cuntinoue. It will take some time. '))
x = []
y = []

pbar = tqdm(total=len(eopatches))
for i in range(len(eopatches)):
    if response != 0:
        break
    eopatch = eopatches[i]
    list_date = []
    for dat in eopatch.timestamp:
        list_date.append(dat.date())
    index = list_date.index(date_)
    
    t = index

    bands = eopatch.data['bands'][t]
    shape = bands.shape[0] * bands.shape[1]
    bands = bands[:,:].flatten().reshape(shape, 13)
    
    weather_mask = eopatch.mask_timeless['weather']
    weather_mask = weather_mask.reshape(shape, 1)
    
    cloud = eopatch.mask['IS_VALID'][t].reshape(shape)
    
    for ind in range(bands.shape[0]):
        weather_date = _weather[_weather['timestamp'] == eopatch.timestamp[t].date()]
        weather = weather_date[weather_date['region'] == weather_mask[ind][0]]
        weather = weather.drop(['timestamp', 'region', 'geometry'],1)
        x_sample = np.append(bands[ind], weather.values)
        x_sample = np.append(cloud[ind], x_sample)
        x_sample = np.append(ind, x_sample)
        #if not cloud[ind]:
        #    continue   
        x.append(x_sample)
        
        #if ind%10000==0:
        #    print(ind)
    
    y_sample = eopatch.mask_timeless['land_cov']
    y_sample = y_sample.reshape(shape)
    
    y.extend(y_sample)
    pbar.update(1)
    
x = np.array(x)

Write 0 if you want to cuntinoue. It will take some time. 1


HBox(children=(IntProgress(value=0, max=7), HTML(value='')))

Wall time: 1.36 s


In [6]:
x = np.array(x)
y = np.array(y)

In [8]:
x.shape, y.shape

((0,), (0,))

In [36]:
x_df = pd.DataFrame(x)
#y_df = pd.DataFrame(y)
x_df['y'] = y
x_df = x_df.sort_values(0,ascending=True)
x_df = x_df[x_df[1] == 1]

In [39]:
y1 = x_df['y'].values
x_df = x_df.drop([0, 1, 'y'], axis=1)
x1 = x_df.values

In [45]:
x_df.head(8)

,2,3,4,5,6,7,8,9,10,11,...,19,20,21,22,23,24,25,26,27,28
0,0.1481,0.1515,0.1280,0.1201,0.1240,0.2412,0.2874,0.2689,0.3113,0.0720,...,66.397386,0.0,0.0,95011.039062,6.799144e+06,15029.321724,0.621561,0.000000,27663.756943,3.140499
224442,0.1205,0.0959,0.0937,0.0677,0.1103,0.2801,0.3440,0.3427,0.3740,0.0906,...,68.657859,0.0,0.0,95491.726562,1.021864e+07,22950.009224,0.156370,0.000000,28263.241318,1.755084
673326,0.1211,0.0947,0.0955,0.0605,0.1069,0.2965,0.3737,0.3449,0.3966,0.0837,...,51.431526,0.0,0.0,98921.976562,9.942890e+06,22836.649849,0.319357,0.000107,30712.710068,4.006664
561105,0.1156,0.0873,0.0734,0.0520,0.0747,0.2010,0.2718,0.2504,0.2834,0.0658,...,61.690616,0.0,0.0,93084.476562,1.051090e+07,22612.509224,0.397077,0.000000,28588.116318,2.757144
561106,0.1156,0.0881,0.0748,0.0526,0.0747,0.2010,0.2718,0.2487,0.2834,0.0658,...,61.690616,0.0,0.0,93084.476562,1.051090e+07,22612.509224,0.397077,0.000000,28588.116318,2.757144
224443,0.1205,0.0968,0.0951,0.0625,0.1159,0.3136,0.3834,0.3643,0.4203,0.0906,...,68.657859,0.0,0.0,95491.726562,1.021864e+07,22950.009224,0.156370,0.000000,28263.241318,1.755084
1,0.1481,0.1491,0.1284,0.0859,0.1396,0.2355,0.2780,0.2917,0.3113,0.0720,...,66.397386,0.0,0.0,95011.039062,6.799144e+06,15029.321724,0.621561,0.000000,27663.756943,3.140499
673327,0.1211,0.0950,0.0964,0.0627,0.1078,0.2893,0.3619,0.3250,0.3771,0.0837,...,51.431526,0.0,0.0,98921.976562,9.942890e+06,22836.649849,0.319357,0.000107,30712.710068,4.006664


In [46]:
x1 = x_df.values

In [47]:
x1

array([[1.48100004e-01, 1.51500002e-01, 1.28000006e-01, ...,
        0.00000000e+00, 2.76637569e+04, 3.14049919e+00],
       [1.20499998e-01, 9.58999991e-02, 9.36999992e-02, ...,
        0.00000000e+00, 2.82632413e+04, 1.75508356e+00],
       [1.21100001e-01, 9.47000012e-02, 9.54999998e-02, ...,
        1.07131898e-04, 3.07127101e+04, 4.00666370e+00],
       ...,
       [1.21500000e-01, 9.61000025e-02, 9.36999992e-02, ...,
        0.00000000e+00, 2.85438819e+04, 1.89444126e+00],
       [1.35000005e-01, 1.00699998e-01, 8.81000012e-02, ...,
        0.00000000e+00, 2.76637569e+04, 3.14049919e+00],
       [1.25599995e-01, 9.83000025e-02, 9.83999968e-02, ...,
        1.07131898e-04, 3.07127101e+04, 4.00666370e+00]])

In [50]:
x = x1
y = y1

In [54]:
y.shape

(562204,)

In [51]:
for i in range(len(y)):
    if y[i] == 2000:
        y[i] = 1
    else:
        y[i] = 0

In [55]:
#np.save('.\\np_data\\bands_weather_x_filtered_and_reordered.npy', x)
#np.save('.\\np_data\\bands_weather_y_filtered_and_reordered.npy', y)

In [56]:
eopatches[0].vector_timeless['weather']

,timestamp,region,10u,10v,2d,2t,rh,sd,sf,sp,ssr,sund,tcc,tp,vis,ws,geometry
0,2017-01-01,0,0.188206,0.383647,257.650900,272.761780,31.846701,0.000000e+00,0.000000,91039.156250,2.652377e+06,13499.990234,0.044308,0.000000,24154.916504,0.656911,"POLYGON ((447003.5777291849 5149253.690335321,..."
1,2017-01-02,0,0.336842,0.459314,258.988152,271.976593,38.423263,0.000000e+00,0.000000,90477.109375,2.665915e+06,13499.974184,0.298159,0.000000,23880.292742,0.902985,"POLYGON ((447003.5777291849 5149253.690335321,..."
2,2017-01-03,0,-0.285087,-0.302480,262.796604,269.972015,58.929505,5.753245e-05,0.000058,90752.523438,2.784197e+06,13612.545898,0.243701,0.000057,23563.123596,0.924396,"POLYGON ((447003.5777291849 5149253.690335321,..."
3,2017-01-04,0,0.856050,-0.002516,262.221876,271.613007,51.419067,1.350045e-05,0.000000,89832.281250,2.651921e+06,13275.000000,0.146483,0.000023,23556.331116,1.306053,"POLYGON ((447003.5777291849 5149253.690335321,..."
4,2017-01-05,0,0.392029,-2.851279,257.301428,267.861061,44.539662,6.485060e-04,0.000660,90162.000000,2.528618e+06,12737.003906,0.193455,0.000661,23467.938965,3.139066,"POLYGON ((447003.5777291849 5149253.690335321,..."
5,2017-01-06,0,-0.573156,-3.396633,251.909365,263.436878,39.019967,9.836629e-06,0.000000,91242.085938,2.594456e+06,13457.510681,0.259747,0.000000,23589.288269,3.573428,"POLYGON ((447003.5777291849 5149253.690335321,..."
6,2017-01-07,0,0.258777,-0.603668,248.962807,264.040592,30.292986,6.718561e-06,0.000000,91246.101562,2.887295e+06,13168.830078,0.286452,0.000000,23442.510437,1.303045,"POLYGON ((447003.5777291849 5149253.690335321,..."
7,2017-01-08,0,0.309214,-0.042710,258.459343,267.454809,49.659060,1.731031e-04,0.000172,90834.375000,1.838917e+06,10350.328125,0.661983,0.000173,22827.512695,0.854073,"POLYGON ((447003.5777291849 5149253.690335321,..."
8,2017-01-09,0,0.540791,-0.104609,261.501396,269.207825,55.778890,6.476417e-06,0.000000,90743.406250,2.432512e+06,13162.500000,0.287591,0.000000,22641.517578,1.127116,"POLYGON ((447003.5777291849 5149253.690335321,..."
9,2017-01-10,0,-0.711287,-0.686248,260.796259,265.456537,70.147793,1.435596e-04,0.000140,90150.468750,2.043844e+06,11181.014406,0.679037,0.000141,19375.680328,1.219086,"POLYGON ((447003.5777291849 5149253.690335321,..."


In [61]:
weather_names_list = list(eopatches[0].vector_timeless['weather'].drop(['timestamp', 'region', 'geometry'],1))
np.save('.\\np_data\\weather_names_list.npy', weather_names_list)

In [3]:
path_out = '.\\eopatches\\'
eopatches = []
for i in range(9):
    eopatches.append(EOPatch.load('{}eopatch_{}'.format(path_out, i), lazy_loading=True))

In [4]:
not_ex = []
index_list = []
for i in range(len(eopatches)):
    eopatch = eopatches[i]
    list_date = []
    date_ = datetime.datetime(2018, 5, 6, 0, 0, 1).month
    
    
    for ind, x in enumerate(eopatch.timestamp):
        list_date.append(x.month)
        if x.month == date_:
            index_list.append((i, ind))

In [5]:
index_list

[(0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (2, 6),
 (2, 7),
 (4, 14),
 (4, 15),
 (4, 16),
 (4, 17),
 (4, 18),
 (5, 10),
 (5, 11),
 (5, 12),
 (5, 13),
 (5, 14),
 (5, 15),
 (6, 14),
 (6, 15),
 (6, 16),
 (6, 17),
 (6, 18),
 (6, 19),
 (6, 20),
 (7, 10),
 (7, 11),
 (7, 12),
 (7, 13),
 (7, 14),
 (8, 10),
 (8, 11),
 (8, 12),
 (8, 13),
 (8, 14)]

In [6]:
%%time
_weather = eopatches[0].vector_timeless['weather']
response = int(input('Write 0 if you want to countinoue. It will take some time. '))
x = []
y = []

pbar = tqdm(total=len(index_list))
for i in index_list:
    print(i)
    if response != 0:
        break
    eopatch = eopatches[i[0]]
    t = i[1]

    bands = eopatch.data['bands'][t]
    shape = bands.shape[0] * bands.shape[1]
    bands = bands[:,:].flatten().reshape(shape, 13)
    
    weather_mask = eopatch.mask_timeless['weather']
    weather_mask = weather_mask.reshape(shape, 1)
    
    cloud = eopatch.mask['IS_VALID'][t].reshape(shape)
    
    for ind in range(bands.shape[0]):
        weather_date = _weather[_weather['timestamp'] == eopatch.timestamp[t].date()]
        weather = weather_date[weather_date['region'] == weather_mask[ind][0]]
        weather = weather.drop(['timestamp', 'region', 'geometry'],1)
        x_sample = np.append(bands[ind], weather.values)
        x_sample = np.append(cloud[ind], x_sample)
        x_sample = np.append(ind, x_sample)
        #if not cloud[ind]:
        #    continue   
        x.append(x_sample)
        
        #if ind%10000==0:
        #    print(ind)
    
    y_sample = eopatch.mask_timeless['land_cov']
    y_sample = y_sample.reshape(shape)
    
    y.extend(y_sample)
    pbar.update(1)
    
x = np.array(x)

Write 0 if you want to cuntinoue. It will take some time. 0


HBox(children=(IntProgress(value=0, max=41), HTML(value='')))

(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(0, 20)
(1, 8)
(1, 9)
(1, 10)
(1, 11)
(2, 6)
(2, 7)
(4, 14)
(4, 15)
(4, 16)
(4, 17)
(4, 18)
(5, 10)
(5, 11)
(5, 12)
(5, 13)
(5, 14)
(5, 15)
(6, 14)
(6, 15)
(6, 16)
(6, 17)
(6, 18)
(6, 19)
(6, 20)
(7, 10)
(7, 11)
(7, 12)
(7, 13)
(7, 14)
(8, 10)
(8, 11)
(8, 12)
(8, 13)
(8, 14)
Wall time: 5h 51min 54s


In [7]:
x = np.array(x)
y = np.array(y)

In [8]:
#np.save('.\\np_data\\bands_weather_x_month.npy', x)
#np.save('.\\np_data\\bands_weather_y_month.npy', y)

In [22]:
x = np.load('.\\np_data\\bands_weather_x_month.npy')
y = np.load('.\\np_data\\bands_weather_y_month.npy')

In [23]:
x_df = pd.DataFrame(x)
x_df['y'] = y
#x_df = x_df.sort_values(0,ascending=True)
x_df = x_df[x_df[1] == 1]

In [24]:
y1 = x_df['y'].values
x_df = x_df.drop([0, 1, 'y'], axis=1)
x1 = x_df.values

In [25]:
for i in range(len(y1)):
    if y1[i] == 2000:
        y1[i] = 1
    else:
        y1[i] = 0

In [26]:
#np.save('.\\np_data\\bands_weather_x_month_mix.npy', x1)
#np.save('.\\np_data\\bands_weather_y_month_mix.npy', y1)

In [11]:
path_out = '.\\eopatches\\'
eopatches = []
for i in range(9):
    eopatches.append(EOPatch.load('{}eopatch_{}'.format(path_out, i), lazy_loading=True))

In [12]:
path_out = '.\\eopatches\\'
eopatches = []
eopatches.append(EOPatch.load('{}eopatch_{}'.format(path_out, 1), lazy_loading=True))

In [13]:
not_ex = []
index_list = []
for i in range(len(eopatches)):
    eopatch = eopatches[i]
    list_date = []
    date_ = datetime.datetime(2018, 5, 6, 0, 0, 1).month
    
    
    for ind, x in enumerate(eopatch.timestamp):
        list_date.append(x.month)
        if x.month == date_:
            index_list.append((i, ind))

In [14]:
not_ex = []
index_list = []
for i in range(len(eopatches)):
    eopatch = eopatches[i]
    list_date = []
    date_ = datetime.datetime(2018, 5, 6, 0, 0, 1).month
    
    
    for ind, x in enumerate(eopatch.timestamp):
        list_date.append(x.month)
        index_list.append((i, ind))

In [15]:
index_list

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (0, 6),
 (0, 7),
 (0, 8),
 (0, 9),
 (0, 10),
 (0, 11),
 (0, 12),
 (0, 13),
 (0, 14),
 (0, 15),
 (0, 16),
 (0, 17),
 (0, 18),
 (0, 19),
 (0, 20),
 (0, 21),
 (0, 22),
 (0, 23),
 (0, 24),
 (0, 25),
 (0, 26),
 (0, 27),
 (0, 28),
 (0, 29),
 (0, 30),
 (0, 31),
 (0, 32),
 (0, 33),
 (0, 34),
 (0, 35),
 (0, 36),
 (0, 37),
 (0, 38),
 (0, 39),
 (0, 40),
 (0, 41),
 (0, 42),
 (0, 43),
 (0, 44),
 (0, 45),
 (0, 46),
 (0, 47),
 (0, 48),
 (0, 49),
 (0, 50),
 (0, 51),
 (0, 52),
 (0, 53),
 (0, 54),
 (0, 55),
 (0, 56),
 (0, 57),
 (0, 58),
 (0, 59),
 (0, 60),
 (0, 61),
 (0, 62),
 (0, 63),
 (0, 64),
 (0, 65),
 (0, 66)]

In [16]:
%%time
_weather = eopatches[0].vector_timeless['weather']
response = int(input('Write 0 if you want to countinoue. It will take some time. '))
x = []
y = []

pbar = tqdm(total=len(index_list))
for i in index_list:
    print(i)
    if response != 0:
        break

    eopatch = eopatches[i[0]]
    t = i[1]

    bands = eopatch.data['bands'][t]
    shape = bands.shape[0] * bands.shape[1]
    bands = bands[:,:].flatten().reshape(shape, 13)

    weather_mask = eopatch.mask_timeless['weather']
    weather_mask = weather_mask.reshape(shape, 1)

    cloud = eopatch.mask['IS_VALID'][t].reshape(shape)

    bands = pd.DataFrame(bands)
    weather_mask = pd.DataFrame(weather_mask, columns = ['we_ma'])
    cloud = pd.DataFrame(cloud, columns = ['cloud'])

    weather_date = _weather[_weather['timestamp'] == eopatch.timestamp[t].date()]

    weather = weather_date[weather_date['region'] == weather_mask['we_ma'][int(bands.shape[0]/2)]]
    weather = weather.drop(['timestamp', 'region', 'geometry'],1)
    weather = weather.append([weather]*bands.shape[0],ignore_index=True)

    x_sample = bands.join(weather)
    x_sample = cloud.join(x_sample)
    x_sample = pd.DataFrame(np.arange(len(x_sample)), columns=['ind']).join(x_sample)
    
    x.extend(x_sample.values)
    
    y_sample = eopatch.mask_timeless['land_cov']
    y_sample = y_sample.reshape(shape)
    
    y.extend(y_sample)
    pbar.update(1)

Write 0 if you want to countinoue. It will take some time. 0


HBox(children=(IntProgress(value=0, max=67), HTML(value='')))

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(0, 5)
(0, 6)
(0, 7)
(0, 8)
(0, 9)
(0, 10)
(0, 11)
(0, 12)
(0, 13)
(0, 14)
(0, 15)
(0, 16)
(0, 17)
(0, 18)
(0, 19)
(0, 20)
(0, 21)
(0, 22)
(0, 23)
(0, 24)
(0, 25)
(0, 26)
(0, 27)
(0, 28)
(0, 29)
(0, 30)
(0, 31)
(0, 32)
(0, 33)
(0, 34)
(0, 35)
(0, 36)
(0, 37)
(0, 38)
(0, 39)
(0, 40)
(0, 41)
(0, 42)
(0, 43)
(0, 44)
(0, 45)
(0, 46)
(0, 47)
(0, 48)
(0, 49)
(0, 50)
(0, 51)
(0, 52)
(0, 53)
(0, 54)
(0, 55)
(0, 56)
(0, 57)
(0, 58)
(0, 59)
(0, 60)
(0, 61)
(0, 62)
(0, 63)
(0, 64)
(0, 65)
(0, 66)
Wall time: 8min


In [17]:
x = np.array(x)
y = np.array(y)

In [18]:
x.shape

(7518807, 29)

In [32]:
x

array([[0, True, 0.15690000355243683, ..., 0.0, 26742.15087890625,
        1.8302086367916348],
       [1, True, 0.15690000355243683, ..., 0.0, 26742.15087890625,
        1.8302086367916348],
       [2, True, 0.15690000355243683, ..., 0.0, 26742.15087890625,
        1.8302086367916348],
       ...,
       [112218, False, 0.0, ..., 0.0, 50423.300384521484,
        1.7931537529614896],
       [112219, False, 0.0, ..., 0.0, 50423.300384521484,
        1.7931537529614896],
       [112220, False, 0.0, ..., 0.0, 50423.300384521484,
        1.7931537529614896]], dtype=object)

In [33]:
x_df = pd.DataFrame(x)
x_df['y'] = y
#x_df = x_df.sort_values(0,ascending=True) #mix
x_df = x_df[x_df[1]] #clouds

In [34]:
y1 = x_df['y'].values
x_df = x_df.drop([0, 1, 'y'], axis=1)
x1 = x_df.values

In [35]:
for i in range(len(y1)):
    if y1[i] == 2000:
        y1[i] = 1
    else:
        y1[i] = 0

In [36]:
np.save('.\\np_data\\x_not_mix.npy', x1)
np.save('.\\np_data\\y_not_mix.npy', y1)

In [37]:
del x1
del y1
del x_df